In [ ]:
#imports
import time
import yfinance as yf
import pandas as pd
import os
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

#manually set data
notebook_name = "Decision Tree Testing Consistency Open Close 3 past tech, finance.ipynb"
VIXMSUSFNvar = ''
maxfeatures = 5 #'sqrt'
GSPCvar = 'all - og'
#all predictors - the original
shrtcomment = 'ConsumeOC'
NYICDXQQQvar = ''
XLKvar = ''
benchmark = '^GSPC'
theticker = 'XLY' #^GSPC
predictionsnum = 'N/A'
longcomment = ''

#settings
dt = 0.6
iterations = 10
number_of_estimators = 1600
minimum_of_samples_split = 150
startnumber = 3000
stepnumber = 220
startdateunformatted = '1950-01-01'

#prep
starting_time = pd.to_datetime(startdateunformatted).date()
today = datetime.now()
tomorrow = today + timedelta(days=1)
tomorrow_str = tomorrow.strftime('%Y-%m-%d')
ending_time = tomorrow_str
tomorrow_str
results = []
predictionfrequency = []
avgpredictionchange = []
avgpredictionchangesd = []
avgweightedpredictionchange = []
avgweightedpredictionchangesd = []
avgweightedbenchmarkchange = []
avgweightedbenchmarkchangesd = []
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)

#data
# Fetch S&P 500 data
sp500 = yf.Ticker(theticker)
sp500 = sp500.history(period="max")

# Ensure the index is a DatetimeIndex
if not isinstance(sp500.index, pd.DatetimeIndex):
    sp500.index = pd.to_datetime(sp500.index)

# Normalize the dates (remove the time component)
sp500.index = sp500.index.normalize()

# Drop columns if needed
if "Dividends" in sp500.columns:
    del sp500["Dividends"]
if "Stock Splits" in sp500.columns:
    del sp500["Stock Splits"]
sp500.index = sp500.index.date
display(sp500)
sp500["Tomorrow"] = sp500["Close"].shift(-1)
######what change were predicting
sp500["Target"] = (sp500["Close"] > sp500["Open"]).astype(int).shift(-1)
#####
sp500 = sp500.loc[starting_time:].copy()
display(sp500)
volatility = yf.download(tickers = '^VIX', start = starting_time, end = ending_time)
volatility = volatility[['Open', 'Adj Close', 'High', 'Low']]
volatility = volatility.rename(columns={'Open': 'VOpen', 'Adj Close': 'VAdj Close', 'High': 'VHigh', 'Low': 'VLow'})
volatility.index = volatility.index.normalize()
volatility.index = volatility.index.date
sp500 = pd.concat([sp500, volatility], axis=1)
USD = yf.download(tickers = '^NYICDX', start = starting_time, end = ending_time)
USD = USD[['Open', 'Adj Close', 'High', 'Low']]
USD = USD.rename(columns={'Open': 'USDOpen', 'Adj Close': 'USDAdj Close', 'High': 'USDHigh', 'Low': 'USDLow'})
USD.index = USD.index.normalize()
USD.index = USD.index.date
sp500 = pd.concat([sp500, USD], axis=1)
#QQQ = yf.download(tickers = 'QQQ', start = starting_time, end = ending_time)
#QQQ = QQQ[['Open', 'Adj Close', 'High', 'Low']]
#QQQ = QQQ.rename(columns={'Open': 'QQQOpen', 'Adj Close': 'QQQAdj Close', 'High': 'QQQHigh', 'Low': 'QQQLow'})
#QQQ.index = QQQ.index.normalize()
#QQQ.index = QQQ.index.date
#sp500 = pd.concat([sp500, QQQ], axis=1)
tech = yf.download(tickers = 'XLK', start = starting_time, end = ending_time)
tech = tech[['Open', 'Adj Close', 'High', 'Low']]
tech = tech.rename(columns={'Open': 'techOpen', 'Adj Close': 'techAdj Close', 'High': 'techHigh', 'Low': 'techLow'})
tech.index = tech.index.normalize()
tech.index = tech.index.date
sp500 = pd.concat([sp500, tech], axis=1)
Consume = yf.download(tickers = '^SP500-25', start = starting_time, end = ending_time)
Consume = Consume[['Open', 'Adj Close', 'High', 'Low']]
Consume = Consume.rename(columns={'Open': 'ConsumeOpen', 'Adj Close': 'ConsumeAdj Close', 'High': 'ConsumeHigh', 'Low': 'ConsumeLow'})
Consume.index = Consume.index.normalize()
Consume.index = Consume.index.date
sp500 = pd.concat([sp500, Consume], axis=1)
msusfn = yf.download(tickers = 'IXF', start = starting_time, end = ending_time)
msusfn = msusfn[['Open', 'Adj Close', 'High', 'Low']]
msusfn = msusfn.rename(columns={'Open': 'msusfnOpen', 'Adj Close': 'msusfnAdj Close', 'High': 'msusfnHigh', 'Low': 'msusfnLow'})
msusfn.index = msusfn.index.normalize()
msusfn.index = msusfn.index.date
sp500 = pd.concat([sp500, msusfn], axis=1)
new_row = pd.DataFrame([[0] * len(sp500.columns)], columns=sp500.columns, index=[pd.to_datetime(tomorrow_str)])
new_row.index = new_row.index.normalize()
new_row.index = new_row.index.date
sp500 = pd.concat([sp500, new_row])
train = sp500.iloc[:-100]
test = sp500.iloc[-100:]

#old
predictors = ["Close", "Volume", "Open", "High"]
#, "Low", "VOpen", "VAdj Close", "VHigh", "VLow"
#predictors = ["Close", "Volume", "Open", "High", "Low"]

#The backtest function evaluates a model by training it on historical data 
#and testing it in increments. It iterates over the data, training the model
#on a growing set of past data and testing it on the next segment. 
#The results are collected and combined into a single DataFrame for analysis.
def backtest(data, model, predictors, start=startnumber, step=stepnumber):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

horizons = [2,5,60,250,1000]
new_predictors = []

for horizon in horizons:
    rolling_averages = sp500.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    
    new_predictors+= [ratio_column, trend_column]

sp500 = sp500.dropna(subset=sp500.columns[sp500.columns != "Tomorrow"])

additional_predictors = ["ConsumeAdj Close", "ConsumeOpen"]
#"msusfnOpen", "msusfnAdj Close"]
    #"USDAdj Close", "USDOpen"] #  "techAdj Close", "techOpen"]# "VOpen", "VAdj Close"] #"VAdj Close"]#, "QQQOpen", "QQQAdj Close"] #["VAdj Close", "VOpen", "VHigh", "VLow"]
#"USDHigh", "USDLow", "USDAdj Close", "USDOpen", ,  "techLow","techHigh", 
#additional_predictors = ["Close", "Volume", "Open", "High", "Low"] #[]#
new_predictors.extend(additional_predictors)

def predict(train, test, predictors, model):
        model.fit(train[predictors], train["Target"])
        preds = model.predict_proba(test[predictors])[:,1]
        preds[preds >=dt] = 1
        preds[preds <dt] = 0
        preds = pd.Series(preds, index=test.index, name="Predictions")
        combined = pd.concat([test["Target"], preds], axis=1)
        return combined

start_time = time.time()
##
benchmarkdata = yf.Ticker(benchmark)
benchmarkdata = benchmarkdata.history(period="max")
### the way change in what time is recorded
benchmarkdata["BMChange"] = ((benchmarkdata["Close"] - benchmarkdata["Open"])/benchmarkdata["Close"]).shift(-1)
###
if not isinstance(benchmarkdata.index, pd.DatetimeIndex):
    benchmarkdata.index = pd.to_datetime(benchmarkdata.index)

benchmarkdata.index = benchmarkdata.index.normalize()
benchmarkdata.index = benchmarkdata.index.date
benchmarkdata = benchmarkdata.drop(benchmarkdata.index[1])
print(benchmarkdata)
#
for i in range (iterations):
    predictionchange = []
    weightedpredictionchange = []
    weightedbmchange = []
    model = RandomForestClassifier(n_estimators=number_of_estimators, max_features=maxfeatures, min_samples_split=minimum_of_samples_split, random_state=i)
    predictions = backtest(sp500, model, new_predictors)
    #display(f"Iteration {i + 1}:")
    #display(predictions)
    #display(predictions["Predictions"].value_counts())
    #display(predictions["Target"].value_counts() / predictions.shape[0])
    #display(np.round(precision_score(predictions["Target"], predictions["Predictions"]), decimals=4))
    results.append(precision_score(predictions["Target"], predictions["Predictions"]))
    
    
    #predictions already happened what happens after is obtaining data
    #newdata
    spy = yf.Ticker(theticker)
    spy = spy.history(period="max")
    #
    if not isinstance(spy.index, pd.DatetimeIndex):
        spy.index = pd.to_datetime(spy.index)
        #
    spy.index = spy.index.normalize()
    del spy["Dividends"]
    del spy["Stock Splits"]
    del spy["Volume"]
    del spy["High"]
    del spy["Low"]
    #### what time for change
    spy["Change"] = (spy["Close"] - spy["Open"]).shift(-1)
    #####
    spy['Open']
    del spy['Close']
    spy.index = spy.index.date
    spy = spy.drop(spy.index[1])
    display(spy)
    total_sum = predictions["Predictions"].value_counts().sum()
    display(total_sum)
    spy = spy.iloc[-total_sum:]
    predictionfrequency.append(predictions["Predictions"].sum() / len(spy))
    
    predictions = predictions.iloc[-total_sum:]    #???total_sum
    change_column = spy['Change'].tolist()
    spy_column = spy['Open'].tolist()
    #display(change_column)
    predictions['Change'] = change_column
    predictions['Weight'] = spy_column
    display(predictions)
    combination = predictions
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    plt.plot(predictions.index, predictions['Predictions'], marker='o', linestyle='-')
    plt.title('Predictions Over Time')
    plt.xlabel('Time')
    plt.ylabel('Predictions')
    plt.grid(True)
    plt.show()
    #display(spy)
    def new_row(row):
        if row['Predictions'] == 0:
            return 0
        else:
            return row['Change']
    predictions['UpdatedChange'] = predictions.apply(new_row, axis=1)
    predictions['WeightedChange'] = predictions['UpdatedChange'] / predictions['Weight']
    display(predictions)
    last_200_rows = predictions.tail(400)
    #display(last_200_rows)
    for value in predictions['UpdatedChange']:
        if value != 0:
            predictionchange.append(value)    
    avgpredictionchange.append(np.mean(predictionchange))
    avgpredictionchangesd.append(np.std(predictionchange))
    for value in predictions['WeightedChange']:
        if value != 0:
            weightedpredictionchange.append(value)    
    avgweightedpredictionchange.append(np.mean(weightedpredictionchange))
    avgweightedpredictionchangesd.append(np.std(weightedpredictionchange))
    ###benchmark stuff
    display(combination)
    combination["BMChange"] = benchmarkdata["BMChange"]
    for idx, value in predictions["UpdatedChange"].items():
        if value != 0:
            weightedbmchange.append(predictions["BMChange"].loc[idx])
        else:
            predictions.at[idx, "BMChange"] = 0
    # Filter out the non-zero elements
    combination["BMChange"] = combination["WeightedChange"] - combination["BMChange"]
    weightedbmchange = combination["BMChange"][combination["BMChange"] != 0]

# Convert the filtered series to a list
    weightedbmchange = weightedbmchange.tolist()

    avgweightedbenchmarkchange.append(np.mean(weightedbmchange))
    avgweightedbenchmarkchangesd.append(np.std(weightedbmchange))
    display(combination)


print(type(combination))
print(type(benchmarkdata))
print(benchmarkdata)

display(results)

end_time = time.time()
elapsed_time = end_time - start_time
display(np.round((elapsed_time / 60), decimals=2))
display(np.round(((elapsed_time / 60) / iterations), decimals=2))

display(np.round(np.mean(avgpredictionchange), decimals=2))
display(np.round(np.mean(avgpredictionchangesd), decimals=4))
display(np.round(np.mean(avgweightedpredictionchange), decimals=6))
display(np.round(np.mean(avgweightedpredictionchangesd), decimals=6))

display(np.round(np.mean(predictionfrequency), decimals=4))
display(np.round(np.mean(results), decimals=4))
display(np.round(np.std(results), decimals=4))

dates_when_predictions_is_1 = predictions.index[predictions['Predictions'] == 1].tolist()

#print(f"All dates when 'predictions' is equal to 1: {dates_when_predictions_is_1}")


# Load the CSV file
filename = "/Users/derek/Downloads/DT AI Transformed Data.csv"
df = pd.read_csv(filename)

# Define your 24 variables (ordered list of values to add)
new_data = [
    notebook_name, number_of_estimators, startdateunformatted, dt, VIXMSUSFNvar, minimum_of_samples_split,
    maxfeatures, GSPCvar, shrtcomment, iterations, NYICDXQQQvar, XLKvar,
    startnumber, stepnumber, theticker, np.round(np.mean(results), decimals=4), np.round(np.std(results), decimals=4), np.round(((elapsed_time / 60) / iterations), decimals=2),
    np.round((elapsed_time / 60), decimals=2), np.round(np.mean(predictionfrequency), decimals = 4), np.round(np.mean(avgpredictionchange), decimals=2), np.round(np.mean(avgpredictionchangesd), decimals=4), np.round(np.mean(avgweightedpredictionchange), decimals=6), np.round(np.mean(avgweightedpredictionchangesd), decimals=6), np.round(np.mean(avgweightedbenchmarkchange), decimals = 6), np.round(np.mean(avgweightedbenchmarkchangesd), decimals=6), longcomment
]

# Find the first empty row
# Here we assume that the row is considered empty if all columns are NaN or empty strings
empty_row_index = df.index[df.isnull().all(axis=1) | (df == '').all(axis=1)].tolist()
if empty_row_index:
    # If there are empty rows, use the first one
    first_empty_row = empty_row_index[0]
else:
    # If no empty rows, append a new row at the end
    first_empty_row = len(df)

# Ensure the length of new_data matches the number of columns
if len(new_data) != len(df.columns):
    raise ValueError("Number of new data values does not match number of columns.")

# Add the new data to the determined row
df.loc[first_empty_row] = new_data

# Save the updated DataFrame back to the CSV file
df.to_csv(filename, index=False)

print(f"Data successfully added to row {first_empty_row + 1} of the CSV file.")

print(f"Number of columns in DataFrame: {len(df.columns)}")
print(f"Number of values in new_data: {len(new_data)}")

print(benchmarkdata)

print(np.mean(avgweightedbenchmarkchange))
print(np.mean(avgweightedbenchmarkchangesd)) 

